---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [3]:
def answer_one():
    total = len(spam_data.index)
    spam = spam_data['target'].value_counts()
    percent = 100*spam[1]/total
    return percent

In [4]:
#answer_one()

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    vect = CountVectorizer().fit(X_train)
    word_lst = vect.get_feature_names()
    longest_token = max(word_lst, key = len)
    return longest_token

In [6]:
#answer_two()

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    vect = CountVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    clfrNB = MultinomialNB(alpha = 0.1).fit(X_train_vectorized, y_train)
    predictions = clfrNB.predict(vect.transform(X_test))
    auc = roc_auc_score(y_test, predictions)
    
    
    return auc

In [8]:
#answer_three()

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    vect = TfidfVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    feature_names = np.array(vect.get_feature_names())
    sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()
    test = sorted(X_train_vectorized.max(0).toarray()[0])
    smallest_tdidf_data = test[:20]
    largest_tdidf_data= test[-1:-21:-1]
    smallest_tdidf_index = feature_names[sorted_tfidf_index[:20]]
    largest_tdidf_index = feature_names[sorted_tfidf_index[-1:-21:-1]]
    smallest_tdidf = pd.Series(data = smallest_tdidf_data, index = [smallest_tdidf_index])
    largest_tdidf = pd.Series(data = largest_tdidf_data, index = [largest_tdidf_index])
    
    return (smallest_tdidf, largest_tdidf)

In [10]:
#answer_four()

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [11]:
def answer_five():
    vect = TfidfVectorizer(min_df = 3).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    clfrNB = MultinomialNB(alpha = 0.1).fit(X_train_vectorized, y_train)
    predictions = clfrNB.predict(vect.transform(X_test))
    auc = roc_auc_score(y_test, predictions)

    return auc

In [12]:
#answer_five()

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [13]:
def answer_six():
    spam_data['length'] = spam_data['text'].str.len()
    spam_class = spam_data[['length','target']].groupby("target").mean()
    
    
    return spam_class['length'][0], spam_class['length'][1]

In [14]:
#answer_six()

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [15]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [16]:
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_seven():
    vect = TfidfVectorizer(min_df = 5).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_train_new = X_train.copy().to_frame()
    X_train_new['length'] = X_train_new['text'].str.len()
    X_train_vectorized_new = add_feature(X_train_vectorized, X_train_new['length'])
    X_test_new = X_test.copy().to_frame()
    X_test_new['length'] = X_test_new['text'].str.len()
    svm = SVC(C = 10000).fit(X_train_vectorized_new, y_train)
    predictions = svm.predict(add_feature(vect.transform(X_test), X_test_new['length']))
    auc = roc_auc_score(y_test, predictions)    
    
    return auc

In [17]:
#answer_seven()

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [18]:
def answer_eight():
    def count_digits(string):
        return sum(item.isdigit() for item in string)
    spam_data['digits'] = spam_data['text'].apply(count_digits)
    spam_digits = spam_data[['digits','target']].groupby("target").mean()

    
    return spam_digits['digits'][0], spam_digits['digits'][1]

In [19]:
#answer_eight()

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [20]:
def answer_nine():
    
    def count_digits(string):
        return sum(item.isdigit() for item in string)
    
    vect = TfidfVectorizer(min_df = 5, ngram_range=(1,3)).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_train_new = X_train.copy().to_frame()
    X_train_new['length'] = X_train_new['text'].str.len()
    X_train_new['digits'] = X_train_new['text'].apply(count_digits)
    X_train_vectorized_new = add_feature(X_train_vectorized, X_train_new['length'])
    X_train_vectorized_new = add_feature(X_train_vectorized_new, X_train_new['digits'])
    
    X_test_new = X_test.copy().to_frame()
    X_test_new['length'] = X_test_new['text'].str.len()
    X_test_new['digits'] = X_test_new['text'].apply(count_digits)
    
    lr = LogisticRegression(C = 100).fit(X_train_vectorized_new, y_train)
    
    X_test_vectorized_new = add_feature(vect.transform(X_test), X_test_new['length'])
    X_test_vectorized_new = add_feature(X_test_vectorized_new, X_test_new['digits'])
    predictions = lr.predict(X_test_vectorized_new)
    auc = roc_auc_score(y_test, predictions)
    
    return auc

In [21]:
#answer_nine()

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [22]:
def answer_ten():
    spam_data['non-words'] = spam_data['text'].str.count(r'\W')
    spam_class = spam_data[['non-words','target']].groupby("target").mean()
    
    
    return (spam_class['non-words'][0], spam_class['non-words'][1])


In [23]:
#answer_ten()

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [24]:
def answer_eleven():
    
    def count_digits(string):
        return sum(item.isdigit() for item in string)
    
    vect = CountVectorizer(ngram_range = (2, 5),analyzer = 'char_wb', min_df = 5).fit(X_train)
    X_train_vectorized = vect.transform(X_train)

    X_train_new = X_train.copy().to_frame()
    X_train_new['length_of_doc'] = X_train_new['text'].str.len()
    X_train_new['digit_count'] = X_train_new['text'].apply(count_digits)
    X_train_new['non-word_char_count'] = X_train_new['text'].str.count(r'\W')
    X_train_vectorized_new = add_feature(X_train_vectorized, [X_train_new['length_of_doc'],X_train_new['digit_count'], X_train_new['non-word_char_count']])
    
    X_test_new = X_test.copy().to_frame()
    X_test_new['length_of_doc'] = X_test_new['text'].str.len()
    X_test_new['digit_count'] = X_test_new['text'].apply(count_digits)
    X_test_new['non-word_char_count'] = X_test_new['text'].str.count(r'\W')

    lr = LogisticRegression(C = 100).fit(X_train_vectorized_new, y_train)
    # get the feature names as numpy array
    feature_names = np.array(vect.get_feature_names())
    feature_names = np.append(feature_names,['length_of_doc', 'digit_count', 'non_word_char_count'])
    
    # Sort the coefficients from the model
    sorted_coef_index = lr.coef_[0].argsort()
    
    
    predictions = lr.predict(add_feature(vect.transform(X_test), [X_test_new['length_of_doc'],X_test_new['digit_count'], X_test_new['non-word_char_count']]))
    auc = roc_auc_score(y_test, predictions)
    test = sorted(X_train_vectorized_new.max(0).toarray()[0])
    smallest_data = test[:10]
    largest_data= test[:-11:-1]
    smallest_index = feature_names[sorted_coef_index[:10]]
    largest_index = feature_names[sorted_coef_index[:-11:-1]]
    smallest_coef = pd.Series(data = smallest_data, index = [smallest_index])
    largest_coef = pd.Series(data = largest_data, index = [largest_index])
    
    return auc, smallest_coef, largest_coef    

In [25]:
#answer_eleven()